In [10]:

df = pd.read_parquet(imdb_path)

# ==========================================
# ADAPTATIONS DES COLONNES
# ==========================================

# 1. Créer un ID numérique si absent
if 'id' not in df.columns:
    df['id'] = range(1, len(df) + 1)

# 2. Renommer les colonnes pour compatibilité
column_mapping = {
'primaryTitle': 'titre',
'averageRating': 'note',
'runtimeMinutes': 'durée',
'numVotes': 'votes'
}

for old_col, new_col in column_mapping.items():
    if old_col in df.columns:
        df[new_col] = df[old_col]

# 3. Convertir genres (string → liste)
    if 'genres' in df.columns:
    df['genre'] = df['genres'].fillna('').apply(
        lambda x: [g.strip() for g in x.split(',')] if isinstance(x, str) and x else []
    )
    else:
    df['genre'] = [[] for _ in range(len(df))]

    # 4. Créer release_date depuis startYear
    if 'startYear' in df.columns:
    df['release_date'] = pd.to_datetime(
        df['startYear'].astype(str) + '-01-01',
        errors='coerce'
    )
    else:
    df['release_date'] = pd.NaT

    # 5. Gérer director.name depuis realisateurs
    if 'realisateurs' in df.columns:
    df['director.name'] = df['realisateurs'].apply(format_director)
    else:
    df['director.name'] = 'Inconnu'

    # 6. Ajouter synopsis (placeholder ou depuis TMDb si disponible)
    if 'synopsis' not in df.columns:
    if 'tmdb_overview_fr' in df.columns:
        df['synopsis'] = df['tmdb_overview_fr'].fillna('Synopsis non disponible.')
    else:
        df['synopsis'] = 'Synopsis non disponible. Consultez IMDb pour plus d\'informations.'

    # 7. Gérer tmdb_id
    if 'tmdb_id' not in df.columns:
    # Créer un ID factice depuis tconst
    df['tmdb_id'] = df['tconst'].apply(lambda x: abs(hash(x)) % 1000000)

    # 8. Calculer a_laffiche (films récents)
    if 'startYear' in df.columns:
    current_year = datetime.now().year
    df['a_laffiche'] = df['startYear'] >= (current_year - 2)  # Films des 2 dernières années
    else:
    df['a_laffiche'] = False

    # 9. Nettoyer les valeurs manquantes
    df['rating'] = pd.to_numeric(df['rating'], errors='coerce').fillna(0)
    df['runtime'] = pd.to_numeric(df['runtime'], errors='coerce').fillna(90)
    df['votes'] = pd.to_numeric(df['votes'], errors='coerce').fillna(0)

    # 10. Filtrer les films de qualité minimale
    df = df[
    (df['rating'] > 0) &
    (df['votes'] >= 100) &
    (df['runtime'] >= 60)
    ].copy()

# ==========================================
# COLONNES FINALES
# ==========================================
colonnes_base = [
'id', 'tconst', 'title', 'release_date', 'genre', 
'runtime', 'rating', 'votes', 'director.name', 
'synopsis', 'tmdb_id', 'a_laffiche'
]

# Ajouter tags et recommandations si elles existent
if 'tags' in df.columns:
colonnes_base.append('tags')
if 'recommandations' in df.columns:
colonnes_base.append('recommandations')

# Garder seulement les colonnes qui existent
colonnes_disponibles = [col for col in colonnes_base if col in df.columns]
df = df[colonnes_disponibles].copy()

# Réinitialiser l'index
df = df.reset_index(drop=True)

df


IndentationError: expected an indented block after 'if' statement on line 24 (1660141040.py, line 25)

In [15]:
import pandas as pd
from pathlib import Path
from datetime import datetime

# Fonction helper
def format_director(realisateurs):
    """Extrait le premier réalisateur"""
    if isinstance(realisateurs, list) and len(realisateurs) > 0:
        return realisateurs[0]
    return 'Inconnu'

# Charger les données
PROJECT_ROOT = Path(r"C:\Users\paulc\Documents\PROJET 2\Cinema_Project2")
imdb_path = PROJECT_ROOT / "data" / "processed" / "imdb_complet_avec_tags"

df = pd.read_parquet(imdb_path)

print(f"📊 Dataset chargé : {len(df):,} lignes")
print(f"📋 Colonnes disponibles : {df.columns.tolist()}\n")

# ==========================================
# ADAPTATIONS DES COLONNES
# ==========================================

# 1. Créer un ID numérique si absent
if 'id' not in df.columns:
    df['id'] = range(1, len(df) + 1)
    print("✅ Colonne 'id' créée")

# 2. Renommer les colonnes pour compatibilité
column_mapping = {
    'primaryTitle': 'title',
    'averageRating': 'rating',
    'runtimeMinutes': 'runtime',
    'numVotes': 'votes'
}

for old_col, new_col in column_mapping.items():
    if old_col in df.columns:
        df[new_col] = df[old_col]
        print(f"✅ Colonne '{old_col}' → '{new_col}'")

# 3. Convertir genres (string → liste)
if 'genres' in df.columns:
    df['genre'] = df['genres'].fillna('').apply(
        lambda x: [g.strip() for g in x.split(',')] if isinstance(x, str) and x else []
    )
    print("✅ Colonne 'genre' créée (liste)")
else:
    df['genre'] = [[] for _ in range(len(df))]
    print("⚠️ Colonne 'genres' absente, liste vide créée")

# 4. Créer release_date depuis startYear
if 'startYear' in df.columns:
    df['release_date'] = pd.to_datetime(
        df['startYear'].astype(str) + '-01-01',
        errors='coerce'
    )
    print("✅ Colonne 'release_date' créée")
else:
    df['release_date'] = pd.NaT
    print("⚠️ Colonne 'startYear' absente")

# 5. Gérer director.name depuis realisateurs
if 'realisateurs' in df.columns:
    df['director.name'] = df['realisateurs'].apply(format_director)
    print("✅ Colonne 'director.name' créée")
else:
    df['director.name'] = 'Inconnu'
    print("⚠️ Colonne 'realisateurs' absente")

# 6. Ajouter synopsis (placeholder ou depuis TMDb si disponible)
if 'synopsis' not in df.columns:
    if 'tmdb_overview_fr' in df.columns:
        df['synopsis'] = df['tmdb_overview_fr'].fillna('Synopsis non disponible.')
        print("✅ Colonne 'synopsis' créée depuis 'tmdb_overview_fr'")
    else:
        df['synopsis'] = 'Synopsis non disponible. Consultez IMDb pour plus d\'informations.'
        print("⚠️ Synopsis par défaut créé")

# 7. Gérer tmdb_id
if 'tmdb_id' not in df.columns:
    # Créer un ID factice depuis tconst
    df['tmdb_id'] = df['tconst'].apply(lambda x: abs(hash(x)) % 1000000)
    print("✅ Colonne 'tmdb_id' créée (hash)")

# 8. Calculer a_laffiche (films récents)
if 'startYear' in df.columns:
    current_year = datetime.now().year
    df['a_laffiche'] = df['startYear'] >= (current_year - 2)  # Films des 2 dernières années
    nb_affiche = df['a_laffiche'].sum()
    print(f"✅ Colonne 'a_laffiche' créée ({nb_affiche:,} films récents)")
else:
    df['a_laffiche'] = False
    print("⚠️ Colonne 'startYear' absente")

# 9. Nettoyer les valeurs manquantes
df['rating'] = pd.to_numeric(df['rating'], errors='coerce').fillna(0)
df['runtime'] = pd.to_numeric(df['runtime'], errors='coerce').fillna(90)
df['votes'] = pd.to_numeric(df['votes'], errors='coerce').fillna(0)
print("✅ Valeurs manquantes nettoyées")

# 10. Filtrer les films de qualité minimale
print(f"\n📊 Avant filtrage : {len(df):,} films")

df = df[
    (df['rating'] > 0) &
    (df['votes'] >= 100) &
    (df['runtime'] >= 60)
].copy()

print(f"📊 Après filtrage : {len(df):,} films")

# ==========================================
# COLONNES FINALES
# ==========================================
colonnes_base = [
    'id', 'tconst', 'title', 'release_date', 'genre', 
    'runtime', 'rating', 'votes', 'director.name', 
    'synopsis', 'tmdb_id', 'a_laffiche'
]

# Ajouter tags et recommandations si elles existent
if 'tags' in df.columns:
    colonnes_base.append('tags')
    print("✅ Colonne 'tags' conservée")
    
if 'recommandations' in df.columns:
    colonnes_base.append('recommandations')
    print("✅ Colonne 'recommandations' conservée")

# Garder seulement les colonnes qui existent
colonnes_disponibles = [col for col in colonnes_base if col in df.columns]
df = df[colonnes_disponibles].copy()

# Réinitialiser l'index
df = df.reset_index(drop=True)

print(f"\n✅ Dataset final : {len(df):,} films, {len(df.columns)} colonnes")
print(f"📋 Colonnes finales : {df.columns.tolist()}\n")

# ==========================================
# AFFICHAGE DES RÉSULTATS
# ==========================================

print("="*60)
print("APERÇU DU DATASET")
print("="*60)

# Afficher les 5 premières lignes
print(df.head())

print("\n" + "="*60)
print("INFORMATIONS SUR LES COLONNES")
print("="*60)
print(df.info())

print("\n" + "="*60)
print("STATISTIQUES DESCRIPTIVES")
print("="*60)
print(df[['rating', 'runtime', 'votes']].describe())

print("\n" + "="*60)
print("EXEMPLES DE GENRES")
print("="*60)
print(df['genre'].head(10))

print("\n" + "="*60)
print("DISTRIBUTION DES FILMS À L'AFFICHE")
print("="*60)
print(df['a_laffiche'].value_counts())

print("\n" + "="*60)
print("TOP 10 FILMS PAR NOTE")
print("="*60)
print(df.nlargest(10, 'rating')[['title', 'rating', 'votes', 'director.name']])

📊 Dataset chargé : 487,439 lignes
📋 Colonnes disponibles : ['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult', 'startYear', 'runtimeMinutes', 'genres', 'averageRating', 'numVotes', 'Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Game-Show', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western', 'acteurs', 'realisateurs', 'tags']

✅ Colonne 'id' créée
✅ Colonne 'primaryTitle' → 'title'
✅ Colonne 'averageRating' → 'rating'
✅ Colonne 'runtimeMinutes' → 'runtime'
✅ Colonne 'numVotes' → 'votes'
✅ Colonne 'genre' créée (liste)
✅ Colonne 'release_date' créée
✅ Colonne 'director.name' créée
⚠️ Synopsis par défaut créé
✅ Colonne 'tmdb_id' créée (hash)
✅ Colonne 'a_laffiche' créée (42,802 films récents)
✅ Valeurs manquantes nettoyées

📊 Avant filtrage : 487,439 films
📊 Après filtrage : 118,277 films
✅ Colonne 'tags'

In [16]:
df

,id,tconst,title,release_date,genre,runtime,rating,votes,director.name,synopsis,tmdb_id,a_laffiche,tags
0,3,tt0035423,Kate & Leopold,2001-01-01,"[Comedy, Fantasy, Romance]",118.0,6.4,93228.0,Inconnu,Synopsis non disponible. Consultez IMDb pour p...,552064,False,Comedy Fantasy Romance Meg-Ryan Hugh-Jackman L...
1,4,tt0036606,"Another Time, Another Place",1983-01-01,"[Drama, War]",118.0,6.4,379.0,Inconnu,Synopsis non disponible. Consultez IMDb pour p...,803043,False,Drama War Phyllis-Logan Giovanni-Mauriello Gia...
2,7,tt0039442,"Habla, mudita",1973-01-01,[Drama],88.0,6.1,181.0,Inconnu,Synopsis non disponible. Consultez IMDb pour p...,361264,False,Drama José-Luis-López-Vázquez Kiti-Mánver Fran...
3,8,tt0044952,Nagarik,1977-01-01,[Drama],127.0,7.9,250.0,Inconnu,Synopsis non disponible. Consultez IMDb pour p...,587,False,Drama Ajit-Banerjee Kali-Bannerjee Gangapada-B...
4,10,tt0053295,Sonnensucher,1971-01-01,[Drama],116.0,6.3,186.0,Inconnu,Synopsis non disponible. Consultez IMDb pour p...,394994,False,Drama Ulrike-von-Zerboni Günther-Simon Erwin-G...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
118272,487416,tt9914942,Life Without Sara Amat,2019-01-01,[Drama],74.0,6.5,257.0,Inconnu,Synopsis non disponible. Consultez IMDb pour p...,720266,False,Drama Biel-Rossell Maria-Morera Francesca-Piñó...
118273,487417,tt9914972,Blind Ambition,2021-01-01,[Documentary],96.0,7.1,330.0,Inconnu,Synopsis non disponible. Consultez IMDb pour p...,316050,False,Documentary Robert-Coe Warwick-Ross 2021
118274,487430,tt9916190,Safeguard,2020-01-01,"[Action, Adventure, Thriller]",95.0,3.6,267.0,Inconnu,Synopsis non disponible. Consultez IMDb pour p...,309354,False,Action Adventure Thriller Patrick-Gallagher Se...
118275,487431,tt9916270,Il talento del calabrone,2020-01-01,[Thriller],84.0,5.8,1536.0,Inconnu,Synopsis non disponible. Consultez IMDb pour p...,440907,False,Thriller Sergio-Castellitto Lorenzo-Richelmy A...


In [25]:
import pandas as pd
path= "C:/Users/paulc/Documents/PROJET 2/Cinema_Project2/data/processed/imdb_complet_avec_cast.parquet"
df_test= pd.read_parquet(path)
df_test[df_test['primaryTitle']== 'Freedom Writers']

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes,...,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,acteurs,realisateurs
104480,tt0463998,movie,Freedom Writers,Freedom Writers,0,2007,123.0,"Biography,Crime,Drama",7.6,91993.0,...,0,0,0,0,0,0,0,0,"[Hilary Swank, Imelda Staunton, Patrick Dempse...",[Richard LaGravenese]
